<a href="https://colab.research.google.com/github/h1chang/ufrgs-tcc-ncm-classifier/blob/main/code/TCC_TFAutoModelForSequenceClassification_NCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
!pip install numpy
!pip install pandas
# !pip install vega_datasets
# !pip install altair
# !pip install geopandas
!pip install transformers datasets evaluate
!pip install datasets

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
# import geopandas as gpd
# import chardet
import matplotlib.pyplot as plt
import os
import nltk
import seaborn as sns
import datasets

# from datasets import DatasetDict
from datasets import Dataset, DatasetDict



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.

# Carregando dados

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# carregando
directory_path = '/content/drive/MyDrive/Colab Notebooks/nf-e'

# todos arquivos da pasta
directory_files = os.listdir(directory_path)

# arquivo específico
directory_files = ['df_capitulo.csv']

df = pd.DataFrame()
for file in directory_files:
    print('Reading file: ', file)
    df_file = pd.read_csv(os.path.join(directory_path, file), encoding = 'ISO-8859-1',low_memory=False)
    df = pd.concat([df, df_file])

# file = '202210_NFe_NotaFiscalItem_novo.csv'
# df_file = pd.read_csv(os.path.join(directory_path, file), encoding = 'ISO-8859-1',low_memory=False)
# df = pd.concat([df, df_file])

print(len(df))

Reading file:  df_capitulo.csv
135221


In [ ]:
# Split the DataFrame into two subsets based on a percentage
df_train = df.sample(frac=0.8, random_state=1)  # First subset with 80% of the data
df_test = df.drop(df_train.index)  # Second subset with the remaining 20% of the data


train_dataset = Dataset.from_dict(df_train)
test_dataset = Dataset.from_dict(df_test)
dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})


In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 108177
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 27044
    })
})

# Login Huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# limitando tamanho do df para executar mais raápido
# dataset_dict["train"] = dataset_dict["train"].shuffle(seed=42).select(range(50))
# dataset_dict["test"] = dataset_dict["test"].shuffle(seed=42).select(range(50))


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/108177 [00:00<?, ? examples/s]

Map:   0%|          | 0/27044 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
# num_epochs = 5
num_epochs = 1
batches_per_epoch = len(tokenized_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
unique_values_train = dataset_dict["train"]["label"]

print(unique_values_train)
print(len(unique_values_train))


unique_values_test = dataset_dict["test"]["label"]
print(unique_values_test)
print(len(unique_values_test))


unique_values = unique_values_train + unique_values_test
unique_values = set(unique_values)

print(unique_values)
print(len(unique_values))

[7, 49, 85, 90, 49, 39, 87, 38, 90, 49, 68, 84, 25, 90, 90, 28, 90, 30, 39, 39, 90, 39, 39, 49, 38, 85, 85, 90, 27, 49, 19, 90, 61, 85, 85, 39, 39, 34, 87, 20, 8, 7, 39, 64, 85, 40, 87, 30, 87, 76, 68, 84, 39, 38, 42, 48, 4, 49, 59, 28, 40, 35, 82, 33, 28, 30, 48, 28, 90, 90, 85, 69, 39, 87, 84, 49, 90, 39, 90, 82, 49, 84, 30, 84, 96, 84, 90, 87, 73, 82, 29, 84, 94, 49, 3, 90, 88, 27, 90, 21, 39, 61, 85, 90, 84, 7, 96, 85, 13, 90, 29, 90, 34, 39, 87, 40, 30, 4, 85, 90, 49, 35, 49, 90, 48, 84, 38, 39, 40, 85, 85, 49, 7, 96, 94, 90, 33, 84, 44, 73, 84, 90, 29, 6, 84, 49, 85, 32, 70, 90, 94, 21, 85, 49, 7, 49, 85, 42, 7, 7, 70, 90, 38, 94, 90, 90, 82, 7, 48, 96, 49, 84, 73, 82, 21, 7, 38, 82, 49, 90, 4, 49, 49, 49, 87, 90, 87, 62, 11, 90, 90, 49, 90, 39, 30, 38, 25, 49, 39, 94, 85, 39, 38, 84, 85, 90, 88, 87, 39, 90, 90, 38, 8, 49, 4, 19, 21, 49, 39, 87, 49, 34, 90, 27, 48, 90, 90, 90, 49, 38, 90, 56, 85, 49, 25, 27, 30, 84, 84, 27, 73, 84, 84, 30, 90, 39, 44, 49, 18, 90, 85, 39, 85, 73, 

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

print(type(id2label))

<class 'dict'>


In [ ]:
# loading id2label from file
id2label = pd.read_csv(os.path.join(directory_path, "id2label - capitulo.csv"), header=None, index_col=0, squeeze = True)

#  convert df to dict
id2label = id2label.to_dict()

# invert dict
label2id = {v: k for k, v in id2label.items()}

print(id2label)
print(len(id2label))

{0: 'DESCONHECIDO', 1: 'Animais vivos.', 2: 'Carnes e miudezas, comestíveis.', 3: 'Peixes e crustáceos, moluscos e outros invertebrados aquáticos.', 4: 'Leite e laticínios; ovos de aves; mel natural; produtos comestíveis de origem animal, não especificados nem compreendidos noutros Capítulos.', 5: 'Outros produtos de origem animal, não especificados nem compreendidos noutros Capítulos.', 6: 'Plantas vivas e produtos de floricultura.', 7: 'Produtos hortícolas, plantas, raízes e tubérculos, comestíveis.', 8: 'Fruta; cascas de citros (citrinos) e de melões.', 9: 'Café, chá, mate e especiarias.', 10: 'Cereais.', 11: 'Produtos da indústria de moagem; malte; amidos e féculas; inulina; glúten de trigo.', 12: 'Sementes e frutos oleaginosos; grãos, sementes e frutos diversos; plantas industriais ou medicinais; palhas e forragens.', 13: 'Gomas, resinas e outros sucos e extratos vegetais.', 14: 'Matérias para entrançar e outros produtos de origem vegetal, não especificados nem compreendidos noutr

<ipython-input-17-e95c450ad318>:2: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  id2label = pd.read_csv(os.path.join(directory_path, "id2label - capitulo.csv"), header=None, index_col=0, squeeze = True)


In [ ]:
from transformers import TFAutoModelForSequenceClassification

# model = TFAutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
# )
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=98, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="tf-distilbert-text-ncm",
    tokenizer=tokenizer,
)

/content/tf-distilbert-text-ncm is already a clone of https://huggingface.co/h1chang/tf-distilbert-text-ncm. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=1, callbacks=callbacks)

5069/6761 [=====================>........] - ETA: 1:21:46 - loss: 1.8142

# Testando classificador

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="h1chang/tf-distilbert-text-ncm")
classifier(text)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("h1chang/tf-distilbert-text-ncm")
inputs = tokenizer(text, return_tensors="tf")

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("h1chang/tf-distilbert-text-ncm")
logits = model(**inputs).logits

In [ ]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]